In [ ]:
!pip install ultralytics kaggle

import os
from google.colab import files
from ultralytics import YOLO
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

print("Настройка Kaggle API")
if not os.path.exists('/root/.kaggle/kaggle.json'):
  print("Загрузите файл kaggle.json (из настроек Kaggle)")
  uploaded = files.upload()

  # Создаем директорию для Kaggle
  os.makedirs('/root/.kaggle', exist_ok=True)

  # Перемещаем файл
  for filename in uploaded.keys():
      os.rename(filename, '/root/.kaggle/kaggle.json')
      os.chmod('/root/.kaggle/kaggle.json', 600)

# Скачивание датасета пластиковых бутылок
print("Скачивание датасета")
!kaggle datasets download -d siddharthkumarsah/plastic-bottles-image-dataset

# Распаковка
!unzip -q -n plastic-bottles-image-dataset.zip -d plastic_bottles_dataset

# Загрузка модели YOLOv8
# https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8n.pt
print("Загрузка модели YOLOv8...")
model = YOLO('yolov8x.pt')
# print(model.names)
print(f"Модель обучена на {len(model.names)} классах")

# Просмотр структуры датасета

print("Структура датасета:")
image_files = glob.glob('plastic_bottles_dataset/**/*.jpg', recursive=True) + \
              glob.glob('plastic_bottles_dataset/**/*.png', recursive=True) + \
              glob.glob('plastic_bottles_dataset/**/*.jpeg', recursive=True)


# image_files = [
#     '/content/plastic_bottles_dataset/Plastic Bottle Image Dataset/train/images/0ca7ac8a475b64ebce575799fffb8c6c_png.rf.ee9bd18643dee88fc258ba975d457a45.jpg',
#     '/content/plastic_bottles_dataset/Plastic Bottle Image Dataset/train/images/000001_jpg.rf.1484582057d57afef76e6966f687fdca.jpg',
#     '/content/plastic_bottles_dataset/Plastic Bottle Image Dataset/train/images/000001_jpg.rf.40ddb746f70899bfc899dc3310aa9dcb.jpg',
#     '/content/plastic_bottles_dataset/Plastic Bottle Image Dataset/test/images/200026_jpg.rf.2dc36abdecf7373657e1fced53869b05.jpg',
# ]

total_images = len(image_files)
# Покажем первые несколько файлов
print(f"Найдено изображений: {total_images}, покажем пути до первых 5:")
for i, file in enumerate(image_files[:5]):
    print(f"  {i+1}. {file}")
print()

###############################################################################
# Тестирование модели на нескольких изображениях
# Берем первые num_demo изображений для демонстрации
num_demo = min(9, total_images)
print(f"Тестирование модели на датасете из {num_demo} изображений:")
test_images = image_files[:num_demo]

# Классы пластиковых объектов
plastic_classes = ['bottle', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'bag']
# plastic_classes = ['bottle']
if num_demo % 3 == 0:
  subplot_row = num_demo // 3
else:
  subplot_row = num_demo // 3 + 1

fig, axes = plt.subplots(subplot_row, 3, figsize=(25, 15))
fig.suptitle('YOLOv8 - Обнаружение пластиковых бутылок в датасете', fontsize=16)

for i, image_path in enumerate(test_images):
    row = i // 3
    col = i % 3

    # Выполняем обнаружение
    results = model(image_path)

    # Анализируем результаты
    plastic_detected = False
    plastic_objects = []

    for box in results[0].boxes:
        class_id = int(box.cls[0])
        class_name = model.names[class_id]
        confidence = float(box.conf[0])

        if any(plastic_word in class_name for plastic_word in plastic_classes) and confidence > 0.3:
            plastic_detected = True
            plastic_objects.append(f"{class_name}: {confidence:.2f}")

    # Визуализируем результат
    result_img = results[0].plot()
    axes[row, col].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))

    # Добавляем заголовок
    status = "Plastic" if plastic_detected else "No plastic"
    title = f"{status}"
    # title = f"{status}\n{os.path.basename(image_path)}"
    if plastic_objects:
        title += f"\n{plastic_objects[0]}"

    axes[row, col].set_title(title, fontsize=10)
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()
print()

###############################################################################
# Анализ более крупной части датасета для статистики обнаружения
num_demo = min(10, total_images)
print(f"Анализ более крупной части датасета для формирования статистики обнаружения ({num_demo} первых изображений):")
bottle_detections = 0
other_plastic_detections = 0
total_detections = 0

for i, image_path in enumerate(image_files[:num_demo]):
    if i % 10 == 0:
        print(f"  Обработано {i}/{num_demo} изображений:")

    results = model(image_path)

    for box in results[0].boxes:
        class_id = int(box.cls[0])
        class_name = model.names[class_id]
        confidence = float(box.conf[0])

        if confidence > 0.3:  # Только уверенные предсказания
            total_detections += 1

            if class_name == 'bottle':
                bottle_detections += 1
            elif any(plastic_word in class_name for plastic_word in plastic_classes):
                other_plastic_detections += 1

print(f"Результаты анализа:")
print(f"  Всего изображений: {num_demo}")
print(f"  Всего обнаружений (любых): {total_detections}")
print(f"  Обнаружено бутылок (с confidence > 0.3): {bottle_detections}")
print(f"  Обнаружено другого пластика: {other_plastic_detections}")
print()
print()


##############################################################################
# Пример реализации системы автоматизации очистки территории от пластика
print("ЗАПУСК СИСТЕМЫ АВТОМАТИЗАЦИИ ОЧИСТКИ ТЕРРИТОРИИ:")
num_demo = min(6, total_images)
CLEANUP_SYSTEMS = {
    'bottle': 'Сборщик бутылок',
    'cup': 'Сборщик стаканов',
    'fork': 'Сборщик приборов',
    'knife': 'Сборщик приборов',
    'spoon': 'Сборщик приборов',
    'bowl': 'Ковшовый сборщик',
    'bag': 'Сетчатый уловитель',
    'other': 'Общий очиститель'
}

# Функция обнаружения загрязнения и запуска системы очистки 
def analyze_and_clean(image_path, model, confidence_threshold=0.3):
    """Анализ изображения и активация очистки"""
    # Поиск объектов загрязнения (пластика)
    results = model(image_path)
    plastic_items = []

    for box in results[0].boxes:
        class_name = model.names[int(box.cls[0])]
        confidence = float(box.conf[0])

        if confidence > confidence_threshold:
            if class_name in plastic_classes:
                plastic_items.append({'class': class_name, 'confidence': confidence})

    # Визуализация
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    img = cv2.imread(image_path)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Оригинал", fontsize=12)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    result_img = results[0].plot()
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))

    if plastic_items:
        plt.title(f"Найдено пластика: {len(plastic_items)}", color='red', fontsize=14)
    else:
        plt.title("Пластик не найден", color='green', fontsize=14)

    plt.axis('off')
    plt.tight_layout()
    plt.show()

    # Активация системы очистки
    if plastic_items:
        print(f"Обнаружено: {len(plastic_items)} пластиковых объектов")
        print("Запуск соответствующей системы очистки: ", end="")

        # Считаем типы объектов
        from collections import Counter
        types_count = Counter([item['class'] for item in plastic_items])

        for obj_type, count in types_count.items():
            system = CLEANUP_SYSTEMS.get(obj_type, CLEANUP_SYSTEMS['other'])
            print(f"{system}: уборка {count} {obj_type}")

        print(f"Очистка завершена")
    else:
        print("Загрязнения не обнаружены. Очистка не требуется")

    return plastic_items

# Запуск анализа
print(f"Анализ датасета из {num_demo} изображений:")

results_summary = []
for i, img_path in enumerate(image_files[:num_demo]):  # Анализируем num_demo изображений
    print()
    print(f"Изображение {i+1}: {os.path.basename(img_path)}")

    try:
        detected = analyze_and_clean(img_path, model, confidence_threshold=0.25)
        plastic_count = len(detected)

        results_summary.append({
            'image': os.path.basename(img_path),
            'plastic_count': plastic_count,
            'status': 'Требует очистки' if plastic_count > 0 else 'Чисто'
        })

        # print(f"Статус: {results_summary[-1]['status']}")

    except Exception as e:
        print(f"Ошибка: {e}")

# Итоги
print()
print("ИТОГИ АНАЛИЗА:")
total_plastic = sum(r['plastic_count'] for r in results_summary)
dirty_areas = sum(1 for r in results_summary if r['plastic_count'] > 0)

print(f"Проанализировано: {len(results_summary)} зон")
print(f"Загрязненных зон: {dirty_areas}")
print(f"Всего пластиковых объектов: {total_plastic}")

if dirty_areas > 0:
    print(f"Система очистки активирована для {dirty_areas} зон")
else:
    print("Все зоны чистые")

print("Система мониторинга завершила работу")